In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")
import ScraperFC as sfc # import local scraperfc
import traceback

from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from ScraperFC import xpath_soup
import re
import requests
import pandas as pd
import numpy as np

print("Done importing.")

In [4]:
sfc.get_proxy()

'45.173.6.5:999'

In [14]:
tm = sfc.Transfermarkt()
try:
    x = tm.get_players(2022, "Eredivisie")
except:
    traceback.print_exc()
finally:
    tm.close()

Gathering player links.
Gathering club links.
Gathering club links.


Gathering player links: 100%|██████████████████████████████████████████████████████████| 18/18 [12:16<00:00, 40.91s/it]
Gathering player https://www.transfermarkt.us/thierry-lutonda/profil/spieler/416802: 100%|█| 655/655 [08:59<00:00,  1.2


In [16]:
x

,Name,Value,Value last updated,DOB,Age,Height (m),Nationality,Citizenship,Position,Other positions,Team,Joined,Contract expires,Market value history,Transfer history
0,Cody Gakpo,€55.00m,"Jun 20, 2023","May 7, 1999",24,1.93,Netherlands,"[Netherlands, Togo]",Left Winger,0 0 Centre-Forward 1 ...,Liverpool FC,"Jan 1, 2023","Jun 30, 2028",None,Season Date Left J...
1,Ulrik Yttergård Jenssen,€500k,"Jun 29, 2023","Jul 17, 1996",27,1.86,Norway,[Norway],Centre-Back,0 0 Defensive Midfield 1 ...,Rosenborg BK,"Jan 8, 2023","Dec 31, 2026",None,Season Date Left ...
2,Mike te Wierik,€400k,"Jun 23, 2023","Jun 8, 1992",31,1.91,Netherlands,[Netherlands],Centre-Back,0 0 Right-Back,FC Emmen,"Feb 1, 2023","Jun 30, 2025",None,Season Date Left ...
3,Zakaria Aboukhlal,€9.00m,"Jun 27, 2023","Feb 18, 2000",23,1.79,Morocco,"[Netherlands, Morocco]",Right Winger,0 0 Left Winger 1 Centre-...,FC Toulouse,"Jul 1, 2022","Jun 30, 2026",None,Season Date Left ...
4,Dimitrios Limnios,€2.00m,"Jun 22, 2023","May 27, 1998",25,1.78,Greece,"[Greece, Brazil]",Right Winger,0 0 Left Winger,1.FC Köln,"Sep 21, 2020","Jun 30, 2024",None,Season Date Left ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650,Mario Götze,€12.00m,"Jun 22, 2023","Jun 3, 1992",31,1.76,Germany,[Germany],Attacking Midfield,0 0 Right Winger 1 Left Winger,Eintracht Frankfurt,"Jul 1, 2022","Jun 30, 2026",None,Season Date Left ...
651,Christopher Mamengi,€350k,"Jul 4, 2023","Apr 3, 2001",22,1.82,Netherlands,"[Netherlands, Angola]",Centre-Back,0 0 Left-Back,Almere City FC,"Aug 17, 2023","Jun 30, 2026",None,Season Date Left ...
652,Rai Vloet,€800k,"Jun 14, 2023","May 8, 1995",28,1.88,Netherlands,[Netherlands],Centre-Forward,0 0 Attacking Midfield 1 ...,Ural Yekaterinburg,"Sep 6, 2022","Dec 31, 2024",None,Season Date Left ...
653,Pontus Almqvist,€1.80m,"Jun 9, 2023","Jul 10, 1999",24,1.83,Sweden,[Sweden],Right Winger,0 0 Left Winger 1 Centre-...,US Lecce,"Jul 1, 2023","Jun 30, 2024",None,Season Date Left ...
